# Kerasのサンプル

バージョン確認

In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.0.6'

MNISTデータのロード

In [2]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

入力をフラットな配列に変換し、学習データは10,000個、テストデータは1000個だけ利用します。また、元データでは濃淡が0から255までで表現されており、最大値が大きすぎるので、特徴の値の最大値を1としておきます。

In [3]:
# input image dimensions
img_rows, img_cols = 28, 28

X_train = X_train.reshape(X_train.shape[0], img_rows * img_cols)[0:10000]
X_test = X_test.reshape(X_test.shape[0], img_rows * img_cols)[0:1000]

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

出力をone-hotに変換します。

In [4]:
from keras.utils import np_utils
Y_train = np_utils.to_categorical(y_train)[0:10000]
Y_test = np_utils.to_categorical(y_test)[0:1000]

## 多層パーセプトロン (MLP)

準備

In [5]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD

まず、中間層1層のモデルを設定します。ユニット数は入力の次元数を考えて、適切に設定してください。活性化関数は、パラメータ[activation](https://keras.io/ja/activations/)で指定します。

In [6]:
input_shape = X_train[0].shape
n_hidden = 200
n_out = len(Y_train[0])  # 10

model = Sequential()
model.add(Dense(n_hidden, input_shape=input_shape, activation='sigmoid'))
model.add(Dense(n_out, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 200)               157000    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                2010      
Total params: 159,010
Trainable params: 159,010
Non-trainable params: 0
_________________________________________________________________


損失関数 ([loss](https://keras.io/ja/losses/))、最適化アルゴリズム([optimizer](https://keras.io/ja/optimizers/))、評価基準([metrics](https://keras.io/ja/metrics/))を指定して、モデルをcompileします。

In [7]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = SGD(lr=0.1),
              metrics = ['accuracy'])

[fit](https://keras.io/ja/models/sequential/#fit)メソッドで学習します。繰り返し回数はパラメータepocksで、バッチサイズはパラメータbatch\_sizeで指定します。

In [8]:
model.fit(X_train, Y_train, epochs=10, batch_size=200)

Epoch 1/10
10000/10000 [==============================] - 0s - loss: 2.0087 - acc: 0.4679     
Epoch 2/10
10000/10000 [==============================] - 0s - loss: 1.4472 - acc: 0.7301     
Epoch 3/10
10000/10000 [==============================] - 0s - loss: 1.0749 - acc: 0.7928     
Epoch 4/10
10000/10000 [==============================] - 0s - loss: 0.8602 - acc: 0.8251     
Epoch 5/10
10000/10000 [==============================] - 0s - loss: 0.7288 - acc: 0.8471     
Epoch 6/10
10000/10000 [==============================] - 0s - loss: 0.6439 - acc: 0.8607     
Epoch 7/10
10000/10000 [==============================] - 0s - loss: 0.5832 - acc: 0.8661     
Epoch 8/10
10000/10000 [==============================] - 0s - loss: 0.5385 - acc: 0.8739     
Epoch 9/10
10000/10000 [==============================] - 0s - loss: 0.5048 - acc: 0.8789     
Epoch 10/10
10000/10000 [==============================] - 0s - loss: 0.4770 - acc: 0.8857     


[evaluate](https://keras.io/ja/models/sequential/#evaluate)メソッドで評価します。

In [9]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.531298254967
Test accuracy: 0.863


### 中間層の層数を増やす

中間層の層数を増やしてみます。その際、活性化関数sigmoid, tanh, reluのそれぞれの違いを確認します。

In [10]:
model = Sequential()
af = 'sigmoid'
model.add(Dense(n_hidden, input_shape=input_shape, activation=af))
model.add(Dense(n_hidden, activation=af))
model.add(Dense(n_hidden, activation=af))
model.add(Dense(n_hidden, activation=af))
model.add(Dense(n_hidden, activation=af))
model.add(Dense(n_out, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 200)               157000    
_________________________________________________________________
dense_4 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_5 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_6 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_7 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_8 (Dense)              (None, 10)                2010      
Total params: 319,810
Trainable params: 319,810
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = SGD(lr=0.1),
              metrics = ['accuracy'])
model.fit(X_train, Y_train, epochs=10, batch_size=200)
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/10
10000/10000 [==============================] - 0s - loss: 2.3154 - acc: 0.1041     
Epoch 2/10
10000/10000 [==============================] - 0s - loss: 2.3091 - acc: 0.1011     
Epoch 3/10
10000/10000 [==============================] - 0s - loss: 2.3098 - acc: 0.1041     
Epoch 4/10
10000/10000 [==============================] - 0s - loss: 2.3092 - acc: 0.1008     
Epoch 5/10
10000/10000 [==============================] - 0s - loss: 2.3103 - acc: 0.1061     
Epoch 6/10
10000/10000 [==============================] - 0s - loss: 2.3081 - acc: 0.1071     
Epoch 7/10
10000/10000 [==============================] - 0s - loss: 2.3079 - acc: 0.1055     
Epoch 8/10
10000/10000 [==============================] - 0s - loss: 2.3107 - acc: 0.1052     
Epoch 9/10
10000/10000 [==============================] - 0s - loss: 2.3091 - acc: 0.1073     
Epoch 10/10
10000/10000 [==============================] - 0s - loss: 2.3079 - acc: 0.0992     
Test loss: 2.3076571579
Test accuracy: 0.099


In [12]:
model = Sequential()
af = 'tanh'
model.add(Dense(n_hidden, input_shape=input_shape, activation=af))
model.add(Dense(n_hidden, activation=af))
model.add(Dense(n_hidden, activation=af))
model.add(Dense(n_hidden, activation=af))
model.add(Dense(n_hidden, activation=af))
model.add(Dense(n_out, activation='softmax'))

In [13]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = SGD(lr=0.1),
              metrics = ['accuracy'])
model.fit(X_train, Y_train, epochs=10, batch_size=200)
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/10
10000/10000 [==============================] - 0s - loss: 0.8224 - acc: 0.7802     
Epoch 2/10
10000/10000 [==============================] - 0s - loss: 0.3739 - acc: 0.8952     
Epoch 3/10
10000/10000 [==============================] - 0s - loss: 0.3104 - acc: 0.9105     
Epoch 4/10
10000/10000 [==============================] - 0s - loss: 0.2728 - acc: 0.9223     
Epoch 5/10
10000/10000 [==============================] - 0s - loss: 0.2511 - acc: 0.9280     
Epoch 6/10
10000/10000 [==============================] - 0s - loss: 0.2274 - acc: 0.9343     
Epoch 7/10
10000/10000 [==============================] - 0s - loss: 0.2100 - acc: 0.9386     
Epoch 8/10
10000/10000 [==============================] - 0s - loss: 0.1931 - acc: 0.9432     
Epoch 9/10
10000/10000 [==============================] - 0s - loss: 0.1825 - acc: 0.9468     
Epoch 10/10
10000/10000 [==============================] - 0s - loss: 0.1659 - acc: 0.9512     
Test loss: 0.283137859583
Test accuracy: 0.906


In [14]:
model = Sequential()
af = 'relu'
model.add(Dense(n_hidden, input_shape=input_shape, activation=af))
model.add(Dense(n_hidden, activation=af))
model.add(Dense(n_hidden, activation=af))
model.add(Dense(n_hidden, activation=af))
model.add(Dense(n_hidden, activation=af))
model.add(Dense(n_out, activation='softmax'))

In [15]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = SGD(lr=0.1),
              metrics = ['accuracy'])
model.fit(X_train, Y_train, epochs=10, batch_size=200)
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/10
10000/10000 [==============================] - 1s - loss: 1.8198 - acc: 0.4443     
Epoch 2/10
10000/10000 [==============================] - 0s - loss: 0.7594 - acc: 0.7511     
Epoch 3/10
10000/10000 [==============================] - 0s - loss: 0.4273 - acc: 0.8684     
Epoch 4/10
10000/10000 [==============================] - 0s - loss: 0.3227 - acc: 0.9045     
Epoch 5/10
10000/10000 [==============================] - 0s - loss: 0.2668 - acc: 0.9183     
Epoch 6/10
10000/10000 [==============================] - 0s - loss: 0.2248 - acc: 0.9347     
Epoch 7/10
10000/10000 [==============================] - 0s - loss: 0.1915 - acc: 0.9422     
Epoch 8/10
10000/10000 [==============================] - 0s - loss: 0.1703 - acc: 0.9485     
Epoch 9/10
10000/10000 [==============================] - 0s - loss: 0.1470 - acc: 0.9558     
Epoch 10/10
10000/10000 [==============================] - 0s - loss: 0.1333 - acc: 0.9618     
Test loss: 0.211743010163
Test accuracy: 0.926


dropoutを入れてみます。学習データとテストデータの正解率の差に注目してください。

In [16]:
from keras.layers.core import Dropout

model = Sequential()
af = 'relu'
model.add(Dense(n_hidden, input_shape=input_shape, activation=af))
model.add(Dropout(0.5))
model.add(Dense(n_hidden, activation=af))
model.add(Dropout(0.5))
model.add(Dense(n_hidden, activation=af))
model.add(Dropout(0.5))
model.add(Dense(n_hidden, activation=af))
model.add(Dropout(0.5))
model.add(Dense(n_hidden, activation=af))
model.add(Dropout(0.5))
model.add(Dense(n_out, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 200)               157000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 200)               40200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 200)               40200     
_________________________________________________________________
dropout_3 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_24 (Dense)             (None, 200)               40200     
__________

In [17]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = SGD(lr=0.1),
              metrics = ['accuracy'])
model.fit(X_train, Y_train, epochs=20, batch_size=200)
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/20
10000/10000 [==============================] - 1s - loss: 2.3199 - acc: 0.1155     
Epoch 2/20
10000/10000 [==============================] - 0s - loss: 2.1915 - acc: 0.1964     
Epoch 3/20
10000/10000 [==============================] - 0s - loss: 1.9650 - acc: 0.3109     
Epoch 4/20
10000/10000 [==============================] - 0s - loss: 1.7746 - acc: 0.3706     
Epoch 5/20
10000/10000 [==============================] - 0s - loss: 1.6383 - acc: 0.4086     
Epoch 6/20
10000/10000 [==============================] - 0s - loss: 1.4871 - acc: 0.4615     
Epoch 7/20
10000/10000 [==============================] - 0s - loss: 1.3441 - acc: 0.5191     
Epoch 8/20
10000/10000 [==============================] - 0s - loss: 1.2231 - acc: 0.5572     
Epoch 9/20
10000/10000 [==============================] - 0s - loss: 1.1375 - acc: 0.5951     
Epoch 10/20
10000/10000 [==============================] - 0s - loss: 1.0562 - acc: 0.6238     
Epoch 11/20
10000/10000 [========================